In [1]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv

In [2]:
# Change directory
%pwd
os.chdir("../")
%pwd

'e:\\MCQ-Generator-using-Generative-Ai'

In [3]:
# Load the Google api key
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=0.2)

c:\Users\HP\anaconda3\envs\ten\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SequentialChain
from langchain_community.callbacks import get_openai_callback


In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [8]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"], # Varible user are going to pass
    template=TEMPLATE
    )


In [9]:
# Quiz chain
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_11356\1298319415.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [10]:
# Check if the quiz is correct or not
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""
# quiz :- output key

In [11]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [12]:
# Review chain
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [13]:
# Connect both the chain
# Generate quiz chain and passing to review chain
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True)

In [ ]:
# Any one method will run (One Pdf)
# Extracting Data from pdf in form of list :- First Method
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("Data/transformer.pdf")
# data = loader.load()  

In [14]:
# Extract Data From the PDF File:- Multiple pdf from a single folder
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",  # Load Only Pdf documents
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [15]:
# Load pdf from the data folder
extracted_data=load_pdf_file(data='Data/')
extracted_data

[Document(metadata={'source': 'Data\\transformer.pdf', 'page': 0, 'page_label': '1'}, page_content="Transformer  \nelectrical transformer is a static electrical machine which transforms electrical power from one circuit to \nanother circuit, without changing the frequency. Transformer can increase or decrease the voltage with \ncorresponding decrease or increase in current. \nWorking principle of transformer \n \n The basic principle behind working of a transformer is the phenomenon of mutual induction between two \nwindings linked by common magnetic flux. The figure at right shows the simplest form of a transformer. \nBasically a transformer consists of two inductive coils; primary winding and secondary winding. The coils are \nelectrically separated but magnetically linked to each other. When, primary winding is connected to a source of \nalternating voltage, alternating magnetic flux is produced around the winding. The core provides magnetic path \nfor the flux, to get linked with t

In [16]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [17]:
NUMBER=5  # Five quiz
SUBJECT="Transformer"
TONE="simple"
TEXT = extracted_data

In [18]:
generate_evaluate_chain

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], input_types={}, partial_variables={}, template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002C3A9C3CEE0>, default_metadata=()), output_key='quiz', output_parser=StrOutputParser(), llm_kwargs={}), LLMChain(verbose=

In [19]:
# Token Usage Tracking in LangChain

with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\HP\AppData\Local\Temp\ipykernel_11356\3383662798.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:[Document(metadata={'source': 'Data\\transformer.pdf', 'page': 0, 'page_label': '1'}, page_content="Transformer  \nelectrical transformer is a static electrical machine which transforms electrical power from one circuit to \nanother circuit, without changing the frequency. Transformer can increase or decrease the voltage with \ncorresponding decrease or increase in current. \nWorking principle of transformer \n \n The basic principle behind working of a transformer is the phenomenon of mutual induction between two \nwindings linked by common magnetic flux. The figure at right shows the simplest form of a transformer. \nBasically a transformer consists of two inductive coils; primary winding and secondary winding. The coils are \nelectrically separated but magnetically linked to each other. When, primary winding is connected to a source of \nalternating voltage, alternating magnet

In [20]:
response

{'text': [Document(metadata={'source': 'Data\\transformer.pdf', 'page': 0, 'page_label': '1'}, page_content="Transformer  \nelectrical transformer is a static electrical machine which transforms electrical power from one circuit to \nanother circuit, without changing the frequency. Transformer can increase or decrease the voltage with \ncorresponding decrease or increase in current. \nWorking principle of transformer \n \n The basic principle behind working of a transformer is the phenomenon of mutual induction between two \nwindings linked by common magnetic flux. The figure at right shows the simplest form of a transformer. \nBasically a transformer consists of two inductive coils; primary winding and secondary winding. The coils are \nelectrically separated but magnetically linked to each other. When, primary winding is connected to a source of \nalternating voltage, alternating magnetic flux is produced around the winding. The core provides magnetic path \nfor the flux, to get link

In [21]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:14942
Prompt Tokens:14076
Completion Tokens:866
Total Cost:0.0


In [22]:
quiz=response.get("quiz")

In [23]:
quiz

'```json\n{\n"1": {\n"mcq": "What is the basic principle behind the working of a transformer?",\n"options": {\n"a": "Electromagnetic repulsion",\n"b": "Mutual induction between two windings",\n"c": "Static electricity",\n"d": "Chemical reactions"\n},\n"correct": "b"\n},\n"2": {\n"mcq": "Which of the following is NOT a type of transformer based on construction?",\n"options": {\n"a": "Core type transformer",\n"b": "Shell type transformer",\n"c": "Ring type transformer",\n"d": "All of the above"\n},\n"correct": "c"\n},\n"3": {\n"mcq": "What is the purpose of the open circuit test on a transformer?",\n"options": {\n"a": "To determine copper losses",\n"b": "To determine core losses",\n"c": "To determine the equivalent resistance",\n"d": "To determine the voltage transformation ratio"\n},\n"correct": "b"\n},\n"4": {\n"mcq": "Why are transformers rated in kVA instead of kW?",\n"options": {\n"a": "Because the current is always AC",\n"b": "Because the losses depend on volt-ampere (VA) and not t

In [24]:
# Remove the leading and trailing Markdown code block formatting (```json and ```)
quiz = quiz.strip().replace('```json', '').replace('```', '')

In [25]:
quiz

'\n{\n"1": {\n"mcq": "What is the basic principle behind the working of a transformer?",\n"options": {\n"a": "Electromagnetic repulsion",\n"b": "Mutual induction between two windings",\n"c": "Static electricity",\n"d": "Chemical reactions"\n},\n"correct": "b"\n},\n"2": {\n"mcq": "Which of the following is NOT a type of transformer based on construction?",\n"options": {\n"a": "Core type transformer",\n"b": "Shell type transformer",\n"c": "Ring type transformer",\n"d": "All of the above"\n},\n"correct": "c"\n},\n"3": {\n"mcq": "What is the purpose of the open circuit test on a transformer?",\n"options": {\n"a": "To determine copper losses",\n"b": "To determine core losses",\n"c": "To determine the equivalent resistance",\n"d": "To determine the voltage transformation ratio"\n},\n"correct": "b"\n},\n"4": {\n"mcq": "Why are transformers rated in kVA instead of kW?",\n"options": {\n"a": "Because the current is always AC",\n"b": "Because the losses depend on volt-ampere (VA) and not the powe

In [26]:
type(quiz)

str

In [27]:
# Convert from str to dict
quiz=json.loads(quiz)

In [28]:
type(quiz)

dict

In [29]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [30]:
quiz_table_data

[{'MCQ': 'What is the basic principle behind the working of a transformer?',
  'Choices': 'a: Electromagnetic repulsion | b: Mutual induction between two windings | c: Static electricity | d: Chemical reactions',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT a type of transformer based on construction?',
  'Choices': 'a: Core type transformer | b: Shell type transformer | c: Ring type transformer | d: All of the above',
  'Correct': 'c'},
 {'MCQ': 'What is the purpose of the open circuit test on a transformer?',
  'Choices': 'a: To determine copper losses | b: To determine core losses | c: To determine the equivalent resistance | d: To determine the voltage transformation ratio',
  'Correct': 'b'},
 {'MCQ': 'Why are transformers rated in kVA instead of kW?',
  'Choices': 'a: Because the current is always AC | b: Because the losses depend on volt-ampere (VA) and not the power factor | c: Because kW is too small a unit | d: To confuse students',
  'Correct': 'b'},
 {'MCQ': '

In [31]:
# Convert to dataframe
quiz=pd.DataFrame(quiz_table_data)

In [32]:
quiz

,MCQ,Choices,Correct
0,What is the basic principle behind the working...,a: Electromagnetic repulsion | b: Mutual induc...,b
1,Which of the following is NOT a type of transf...,a: Core type transformer | b: Shell type trans...,c
2,What is the purpose of the open circuit test o...,a: To determine copper losses | b: To determin...,b
3,Why are transformers rated in kVA instead of kW?,a: Because the current is always AC | b: Becau...,b
4,Under what condition is the efficiency of a tr...,a: When the load is at its maximum | b: When t...,b


In [33]:
quiz.to_csv("data/quiz.csv",index=False) # save in data folder